In [70]:
###Dependecies
import sqlite3
import pandas as pd
from io import BytesIO
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [4]:
steam_games_df = pd.read_csv("db_files/steam_games.csv", encoding = 'utf-8', dtype={'name':str})
steam_games_df.head()

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN


In [5]:
#remove uneeded rows
delete_labels = ['url', 'types', 'desc_snippet', 'popular_tags', 'game_details', 'languages', 'achievements',
                 'game_description', 'mature_content', 'minimum_requirements', 'recommended_requirements', 'discount_price']
steam_games_df = steam_games_df.drop(columns=delete_labels)
steam_games_df.head()

,name,recent_reviews,all_reviews,release_date,developer,publisher,genre,original_price
0,DOOM,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks",Action,$19.99
1,PLAYERUNKNOWN'S BATTLEGROUNDS,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Action,Adventure,Massively Multiplayer",$29.99
2,BATTLETECH,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Action,Adventure,Strategy",$39.99
3,DayZ,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Action,Adventure,Massively Multiplayer",$44.99
4,EVE Online,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Action,Free to Play,Massively Multiplayer,RPG,...",Free


In [6]:
steam_games_df['name'] = steam_games_df['name'].astype('str')
steam_games_df['name'].str.encode('ascii', 'ignore').str.decode('ascii')
#steam_games_df['name'].str.find('Kingdom Come: Deliverance').value_counts()

steam_games_df.loc[steam_games_df['name'] == 'Kingdom Come: Deliverance']

#mask_nonAscii = steam_games_df['name'].map(str.isascii)
#mask_nonAscii.value_counts()

,name,recent_reviews,all_reviews,release_date,developer,publisher,genre,original_price
352,Kingdom Come: Deliverance,"Mostly Positive,(450),- 72% of the 450 user re...","Mostly Positive,(27,512),- 74% of the 27,512 u...","Feb 13, 2018",Warhorse Studios,"Warhorse Studios,Deep Silver,Warhorse Studios,...","Action,Adventure,RPG",$29.99


In [7]:
metacritic_df = pd.read_csv("db_files/metacritic_games.csv", encoding = 'utf-8')
metacritic_df

,game,platform,developer,genre,number_players,rating,release_date,positive_critics,neutral_critics,negative_critics,positive_users,neutral_users,negative_users,metascore,user_score
0,Portal 2,PC,Valve Software,Action,NaN,E10+,"Apr 18, 2011",51,1,0,1700,107,19,95,90
1,The Elder Scrolls V: Skyrim,PC,Bethesda Game Studios,Role-Playing,No Online Multiplayer,M,"Nov 10, 2011",32,0,0,1616,322,451,94,82
2,The Legend of Zelda: Ocarina of Time 3D,3DS,GREZZO,Miscellaneous,No Online Multiplayer,E10+,"Jun 19, 2011",84,1,0,283,20,5,94,90
3,Batman: Arkham City,PC,Rocksteady Studios,Action Adventure,NaN,T,"Nov 21, 2011",27,0,0,240,34,27,91,87
4,Super Mario 3D Land,3DS,Nintendo,Action,No Online Multiplayer,E,"Nov 13, 2011",81,1,0,251,39,11,90,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5694,Modern Combat: Blackout,Switch,Gameloft,Action,No Online Multiplayer,NaN,"Feb 14, 2019",0,5,2,1,0,1,53,57
5695,Fimbul,PS4,Zaxis,Action Adventure,No Online Multiplayer,NaN,"Feb 28, 2019",0,7,2,1,0,0,52,18
5696,Generation Zero,PC,Avalanche Studios,Action Adventure,Online Multiplayer,T,"Mar 26, 2019",1,11,6,5,2,6,50,57
5697,Vane,PS4,Friend & Foe,Adventure,No Online Multiplayer,E10+,"Jan 15, 2019",3,26,22,2,2,2,49,55


In [8]:
kickstarter_df = pd.read_csv("db_files/ks-projects-201801.csv", encoding = 'utf-8')
kickstarter_df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [9]:
#considering taking this out and doing more web scraping to get relevant values

#sort kickstarter df for just video games
ks_games_df = kickstarter_df.loc[kickstarter_df['category']=='Video Games']
#sort for successful ks campaigns
ks_games_df = ks_games_df.loc[ks_games_df['state'] == 'successful']
ks_games_df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
126,1000524949,Operation: Make Stuff,Video Games,Games,USD,2012-10-18,200.0,2012-09-18 04:46:06,306.72,successful,36,US,306.72,306.72,200.00
199,1000811882,The Sword of Asumi Visual Novel,Video Games,Games,GBP,2014-11-10,2500.0,2014-10-19 20:35:23,3097.00,successful,107,GB,4983.78,4848.23,3913.65
219,1000940032,Space Trucker,Video Games,Games,USD,2016-12-08,150.0,2016-11-08 08:20:27,165.00,successful,7,US,25.00,165.00,150.00
319,1001442379,The Contractors - Create 3D Model: Prop Asset ...,Video Games,Games,USD,2017-05-26,2000.0,2017-05-05 12:47:05,2348.00,successful,17,US,340.00,2348.00,2000.00
475,1002272545,Course: Make a Stunning Fantasy RPG with GameM...,Video Games,Games,USD,2016-08-13,10200.0,2016-07-14 00:01:11,20709.50,successful,1258,US,11973.50,20709.50,10200.00


In [10]:
ign_df = pd.read_csv("db_files/IGN_reviews.csv", encoding = 'utf-8')
ign_df.head()

,Game,Platform,Score,Genre
0,Wolfenstein: The New Order,Xbox One,7.8,Shooter
1,Mario Kart 8,Wii U,9.0,"Racing, Action"
2,Sportsfriends,PlayStation 3,8.7,"Action, Compilation"
3,Sportsfriends,PlayStation 4,8.7,"Action, Compilation"
4,Sportsfriends,PC,8.7,"Action, Compilation"


In [68]:
sorted_ign_df = pd.DataFrame(columns=['Game', 'Platform', 'Score', 'Genre'])
for index, row in ign_df:
    if

SyntaxError: unexpected EOF while parsing (<ipython-input-68-89737371ff37>, line 2)

In [11]:
#Now to scrape the kickstarter page for successful games that launched on steam
#looking for the names of each game and potentially other info that might be interesting to add to the db
url='https://www.kickstarter.com/play?sort=alpha#all-projects'
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\pwitw\.wdm\drivers\chromedriver\win32\87.0.4280.88]


In [68]:
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
steam_links=[]

In [69]:
counter = 0
while soup.find('a', class_='more btn btn--border-gray', href=True) is not None:
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    try:
        button = soup.find('a', class_='more btn btn--border-gray', href=True)['href']
        browser.visit(button)
        counter += counter
        print(f"clicks: {counter}")
        time.sleep(1)
    except TypeError:
        pass

clicks: 0
clicks: 0
clicks: 0
clicks: 0
clicks: 0
clicks: 0
clicks: 0
clicks: 0
clicks: 0
clicks: 0
clicks: 0
clicks: 0
clicks: 0
clicks: 0
clicks: 0
clicks: 0


In [61]:
button = soup.find('a', class_='more btn btn--border-gray', href=True)['href']
browser.visit(button)

In [43]:

ks_pages = soup.find('section', {'id' : 'all-projects'})
for a in ks_pages.find_all('a', class_= 'hover-zoomout f3 bg-green-90 track-service', href=True):
    if a['href'] not in steam_links:
        steam_links.append(a['href'])


['https://store.steampowered.com/app/251570', 'https://store.steampowered.com/app/280040', 'https://store.steampowered.com/app/300280', 'https://store.steampowered.com/app/230310', 'https://store.steampowered.com/app/628290', 'https://store.steampowered.com/app/250620', 'https://store.steampowered.com/app/380750', 'https://store.steampowered.com/app/269890', 'https://store.steampowered.com/app/290340', 'https://store.steampowered.com/app/334120', 'https://store.steampowered.com/app/816560', 'https://store.steampowered.com/app/204300', 'https://store.steampowered.com/app/237470', 'https://store.steampowered.com/app/395850', 'https://store.steampowered.com/app/308060']


In [40]:
ks_pages = soup.find('section', {'id' : 'all-projects'})
for a in ks_pages.find_all('a', class_= 'hover-zoomout f3 bg-green-90 track-service', href=True):
    try:
        #print (f"Found the URL: "  + a['href'])
        steam_links.append(a['href'])
    except KeyError:
        pass
    
#time.sleep(0.1)

Found the URL: https://store.steampowered.com/app/251570
Found the URL: https://store.steampowered.com/app/280040
Found the URL: https://store.steampowered.com/app/300280
Found the URL: https://store.steampowered.com/app/230310
Found the URL: https://store.steampowered.com/app/628290
Found the URL: https://store.steampowered.com/app/250620
Found the URL: https://store.steampowered.com/app/380750
Found the URL: https://store.steampowered.com/app/269890
Found the URL: https://store.steampowered.com/app/290340
Found the URL: https://store.steampowered.com/app/334120
Found the URL: https://store.steampowered.com/app/816560
Found the URL: https://store.steampowered.com/app/204300
Found the URL: https://store.steampowered.com/app/237470
Found the URL: https://store.steampowered.com/app/395850
Found the URL: https://store.steampowered.com/app/308060


In [64]:
browser.is_element_present_by_text('View project')
project_links = browser.links.find_by_partial_text('View project')
project_links
base_url = 'https://www.kickstarter.com/'
for link in project_links:
    project_url = project_links[link]
    browser.visit(base_url + project_url)
    
    time.sleep(0.1)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=87.0.4280.88)


In [ ]:
test_title = soup.find('div', class_='NS_site__spotlight_project') 
found_title = test_title.find('div', class_='content_title').text
found_title
